In [24]:
!pip install --upgrade torch onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 86.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 74.7 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: onnx
    Found existing installation: onnx 1.9.0
    Uninstalling onnx-1.9.0:
      Successfully uninstalled onnx-1.9.0
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip uninstall hifi_gan -y

Found existing installation: hifi-gan 0.0.2
Uninstalling hifi-gan-0.0.2:
  Successfully uninstalled hifi-gan-0.0.2


In [16]:
!pip install -e hifi_gan

Obtaining file:///usr/src/app/hifi_gan
  Preparing metadata (setup.py) ... done
  Attempting uninstall: hifi-gan
    Found existing installation: hifi-gan 0.0.2
    Uninstalling hifi-gan-0.0.2:
      Successfully uninstalled hifi-gan-0.0.2
  Running setup.py develop for hifi-gan
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
hifi_gan_config = {
    "resblock": "1",
    "num_gpus": 0,
    "batch_size": 16,
    "learning_rate": 0.0002,
    "adam_b1": 0.8,
    "adam_b2": 0.99,
    "lr_decay": 0.999,
    "seed": 1234,
    "upsample_rates": [8, 8, 2, 2],
    "upsample_kernel_sizes": [16, 16, 4, 4],
    "upsample_initial_channel": 512,
    "resblock_kernel_sizes": [3, 7, 11],
    "resblock_dilation_sizes": [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
    "segment_size": 8192,
    "num_mels": 80,
    "num_freq": 1025,
    "n_fft": 1024,
    "hop_size": 256,
    "win_size": 1024,
    "sampling_rate": 22050,
    "fmin": 0,
    "fmax": 8000,
    "fmax_for_loss": None,
    "num_workers": 4,
    "dist_config": {
        "dist_backend": "nccl",
        "dist_url": "tcp://localhost:54321",
        "world_size": 1,
    },
}

In [3]:
!ls

 1.wav		   docker		 onnx.ipynb
 CONTRIBUTING.md   docker-compose.yml	 scripted-tacotron2.pt
 LICENSE	   docs			 settings.ini
 LICENSE2	   hifi-gan-onnx.ipynb	 setup.py
 LICENSE3	   hifi_gan		 test.onnx
 LICENSE4	   hifi_gan.onnx	'torchscript jit-Copy1.ipynb'
 MANIFEST.in	  'mel_out (1).json'	'torchscript jit.ipynb'
 Makefile	  'mel_out (2).json'	 train.sh
 README.md	   mel_out.json		 uberduck_ml_dev
 applications	   model_store		 uberduck_ml_dev.egg-info
 build_lib	   monotonic_align
 configs	   nbs


In [8]:
from IPython.display import Audio

In [10]:
Audio("1.wav")

In [4]:
import torch
from uberduck_ml_dev.models.common import griffin_lim, MelSTFT
from hifi_gan.models import Generator
from hifi_gan.env import AttrDict

In [5]:
stft = MelSTFT()

In [6]:
from scipy.io.wavfile import read, write

In [7]:
sr, data = read("1.wav")

<ipython-input-7-8085c45d337f>:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, data = read("1.wav")


In [8]:
data

array([-3, -2, -2, ...,  3,  0,  1], dtype=int16)

In [9]:
data.min(), data.max()

(-15010, 14862)

In [10]:
normalized_data = data / 32768.0

In [11]:
import torch
tensor_data = torch.FloatTensor(normalized_data)

In [12]:
mel = stft.mel_spectrogram(tensor_data.unsqueeze(0))

In [13]:
state_dict = torch.load("model_store/g_02590000_8spk", map_location="cpu")
config = AttrDict(hifi_gan_config)
generator = Generator(config)
generator.load_state_dict(state_dict["generator"])

<All keys matched successfully>

In [14]:
generator.eval()
with torch.no_grad():
    output = generator(mel)

In [15]:
print(output.shape)

torch.Size([1, 1, 77312])


In [16]:
output = output.squeeze()

In [17]:
output.shape

torch.Size([77312])

In [18]:
audio = (output * 32768).numpy().astype("int16")

In [59]:
Audio(audio, rate=22050)

In [60]:
mel.shape

torch.Size([1, 80, 302])

In [34]:
generator.eval()
torch.onnx.export(
    generator,
    mel,
    "hifi_gan.onnx", 
    verbose=True, 
    opset_version=15,
    dynamic_axes=dict(
        OUTPUT_0={0: "batch_size", 2: "audio_sequence_length"},
        INPUT_0={0: "batch_size", 2: "mel_sequence_length"},
    ),
    input_names=["INPUT_0"],
    output_names=["OUTPUT_0"] # , "output_lengths", "output_speaker_id", "output_gst"],
    # output_names=["mel_out"] # , "mel_postnet", "gate", "attn"]# , "mel_lenghts"], #, "output_frames"],
)

graph(%INPUT_0 : Float(*, 80, *, strides=[24160, 302, 1], requires_grad=0, device=cpu),
      %conv_pre.bias : Float(512, strides=[1], requires_grad=1, device=cpu),
      %ups.0.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %ups.1.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %ups.2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %ups.3.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %resblocks.0.convs1.0.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %resblocks.0.convs1.1.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %resblocks.0.convs1.2.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %resblocks.0.convs2.0.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %resblocks.0.convs2.1.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %resblocks.0.convs2.2.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %

In [35]:
import onnx

In [36]:
onnx_model = onnx.load("hifi_gan.onnx")

In [37]:
onnx.checker.check_model(onnx_model)

In [38]:
import onnxruntime as ort

In [39]:
ort_session = ort.InferenceSession("hifi_gan.onnx")

In [40]:
outputs= ort_session.run(None, dict(
    INPUT_0=mel.numpy(),
))

In [41]:
outputs

[array([[[0.00020893, 0.00011616, 0.00012352, ..., 0.00012595,
          0.0001541 , 0.00020127]]], dtype=float32)]

In [42]:
audio = (outputs[0][0] * 32786).astype("int16")

In [43]:
audio

array([[6, 3, 4, ..., 4, 5, 6]], dtype=int16)

In [44]:
from IPython.display import Audio

Audio(audio, rate=22050)

In [46]:
!ls -lash hifi_gan.onnx

54M -rw-r--r-- 1 root root 54M Jun 16 03:18 hifi_gan.onnx
